<a href="https://colab.research.google.com/github/raman2-dev/amazon-web-scraping-python-project/blob/main/amazon_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [31]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-price-whole'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [32]:

if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=smartphone+under+40000&rh=n%3A976419031%2Cp_36%3A1318507031&dc&ds=v1%3A0Em9lTah8Ec1rRrsBzq%2BUqx3X7DclV9jmlq7%2BWrKtuc&rnid=1318502031&ref=sr_nr_p_36_5"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [33]:
amazon_df

,title,price,rating,reviews,availability
0,"Samsung Galaxy A35 5G (Awesome Lilac, 8GB RAM,...","₹33,999.00",3.2 out of 5 stars,25,In stock
1,"Samsung Galaxy S20 FE 5G (Cloud Mint, 8GB RAM,...","₹28,999.00",4.2 out of 5 stars,"35,281 ratings",In stock
2,"Samsung Galaxy S22 5G (Green, 8GB, 128GB Stora...","₹37,889.00",4.3 out of 5 stars,"1,794",In stock
3,"iQOO Z7 Pro 5G (Blue Lagoon, 8GB RAM, 256GB St...","₹24,999.00",4.4 out of 5 stars,"10,190 ratings",In stock
4,"HONOR 90 (Emerald Green, 8GB + 256GB) | India'...","₹25,999.00",4.0 out of 5 stars,"2,850",In stock
5,"realme GT Neo 3 (150W) (Sprint White, 12GB RAM...","₹24,100.00",4.1 out of 5 stars,105 ratings,In stock
6,"HONOR X9b 5G (Midnight Black, 8GB + 256GB) | I...","₹25,999.00",4.0 out of 5 stars,446,In stock
7,"Samsung Galaxy A34 5G (Awesome Graphite, 8GB, ...","₹24,499.00",4.1 out of 5 stars,"1,975",In stock
8,"Samsung Galaxy A34 5G (Awesome Silver, 8GB, 12...","₹24,499.00",4.1 out of 5 stars,"1,975",In stock
9,"HONOR X9b 5G (Sunrise Orange, 8GB + 256GB) | I...","₹25,999.00",4.0 out of 5 stars,446 ratings,In stock
